In [199]:
import numpy as np
import random as rdm

In [248]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def relu(X):
    return np.array([max(0,x) for x in X])

def sigmoid_prime(x):
    return sigmoid(x)*(1-sigmoid(x))

def relu_prime(X):
    return np.array([1 if x >= 0 else 0 for x in X])

def activation(name):
    if name =='sigmoid':
        return sigmoid,sigmoid_prime
    elif name=='relu':
        return relu,relu_prime
    else:
        raise Exception('activation function not in [sigmoid,relu]')
        
def mean_square_error(y_hat,y):
    return np.square(y_hat - y).mean(axis = 1)

def mean_square_error_der(y_hat,y):
    if len(y) > 1:
        return 2*(y_hat - y).mean(axis = 1)
    else:
        return 2*(y_hat - y)

def square_sum(X):
    return np.square(X).sum()

def loss_function(name):
    if name=='mean_square_error':
        return mean_square_error , mean_square_error_der
    else:
        raise Exception('loss function not in [mean_square_error]')

def penalization(name):
    if name == 'square_sum':
        return square_sum
    else:
        raise Exception('penalization function not in [square_sum]')

In [297]:
class NeuralNetwork:
    def __init__(self,layers_parameters , global_parameters):
        self.layers = {l:{'W' : 0.1*np.random.random(size = (layers[l]['output_dim'],layers[l]['input_dim'])),
                          'B' : 0.1*np.random.random(size = (layers[l]['output_dim'])),
                          'g' : activation(layers[l]['activation'])[0],
                          'g_prime': activation(layers[l]['activation'])[1]
                         } for l in layers}
        
        self.input_dim = global_parameters['input']
        self.output_dim = global_parameters['output']
        self.layers_dim = global_parameters['layers']
        self.batch_size = global_parameters['batch']
        self.lr = global_parameters['learning_rate']
        
        self.loss = loss_function(global_parameters['loss'])[0]
        self.loss_der = loss_function(global_parameters['loss'])[1]
        
    def predict(self,X):
        if len(X) != self.input_dim:
            raise Exception('input of size {} instead of {}'.format(len(X),self.input_dim))
        for n in range(self.layers_dim):
            X = self.layers[n]['g'](np.dot(self.layers[n]['W'] , X) + self.layers[n]['B'])
        return X
    
    def loss_calculation(self,x,y):
        y_hat = self.predict(X)
        return self.loss(y_hat,h)
    
    def forward(self,x):
        intermediary_step = []
        if len(x) != self.input_dim:
            raise Exception('input of size {} instead of {}'.format(len(x),self.input_dim))
        
        for n in range(self.layers_dim):
            z = np.dot(self.layers[n]['W'] , x) + self.layers[n]['B']
            h = self.layers[n]['g'](z)
            intermediary_step.append([x , z , h])
            x = np.array(h)
            
        return intermediary_step
    
    def backward(self , intermediary_step , y):
        if len(intermediary_step) != self.layers_dim:
            raise Exception('dimensions do not match')
        
        print(intermediary_step[-1][2],y)
        last_gradient_h = self.loss_der(intermediary_step[-1][2] , y)
        
        for i in range(len(intermediary_step)):
            n = len(intermediary_step) - i - 1
            x , z , h = intermediary_step[n]
            last_gradient_z = np.dot(last_gradient_h , self.layers[n]['g_prime'](z))
            print(x)
            x = x.reshape((2,1))
            print(x.T)
            last_gradient_w = np.dot(last_gradient_z , x.T)
            last_gradient_b = last_gradient_z
            last_gradient_h = np.dot(last_gradient_z,self.layers[n]['W'])
            
            self.layers[n]['W'] -= self.lr * last_gradient_w
            self.layers[n]['B'] -= self.lr * last_gradient_b
        
        return
    
    def train(self , X , Y):
        if len(X) != len(Y):
            raise Exception('X and Y dimension do not match')
        
        for i in range(len(X)):
            x , y = X[i] , Y[i]
            intermediary_step = self.forward(x)
            self.backward(intermediary_step,np.array([y]))  
        return score(self,X,Y)
    
    def score(self,X,Y):
        mean_square = 0
        for i in range(len(X)):
            y = self.predict(X[i])
            mean_square += np.square(y - Y[i])
        return mean_square

In [298]:
layers = {
          0:{'input_dim' : 2,
             'output_dim': 2,
             'activation': 'relu'},
          1:{'input_dim' : 2,
             'output_dim': 1,
             'activation': 'relu'}
         }

parameters = {'input': 2,
              'output' : 1,
              'layers': 2,
              'loss': 'mean_square_error',
              'penalization':'square_sum',
              'batch':1,
              'learning_rate':0.1}

nn = NeuralNetwork(layers , parameters)

In [299]:
data = []
for i in range(10000):
    y = i%2
    dist = np.random.normal(loc = 1 + i%2,scale = 0.2)
    x0 = np.random.uniform(low = -np.sqrt(dist/2),high = np.sqrt(dist/2))
    sign = -1 if np.random.random() > 0.5 else 1
    x1 = sign*np.sqrt(dist - x0**2)
    data.append([x0,x1,y])
X = np.array(data)[:,:2]
Y = np.array(data)[:,2]

In [300]:
nn.train(X,Y)

[0.07995246] [0.]
[0.04349843 0.01379485]
[[0.04349843 0.01379485]]
[ 0.35315023 -0.64082286]
[[ 0.35315023 -0.64082286]]


ValueError: shapes (1,) and (2,2) not aligned: 1 (dim 0) != 2 (dim 0)